In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from psycopg2 import Error
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas


In [27]:
db_params = {
    'host': 'localhost',
    'port': '5432',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'whoami'
}

try:
    connection = psycopg2.connect(**db_params)
    print("Successfully connected to the database")

    query = "SELECT * FROM cases"
    df = pd.read_sql_query(query, connection)

    df['case_created_at'] = pd.to_datetime(df['case_created_at'])

    connection.close()
    print("Database connection closed")

except Error as e:
    print(f"Error connecting to PostgreSQL: {e}")
    exit()

Successfully connected to the database
Database connection closed


/var/folders/fq/f_p95hfx63j6_m9f116tk0p40000gn/T/ipykernel_28588/1283296435.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [28]:
if df.empty:
    print("No data found")

df


,case_id,user_id,comment,case_created_at,case_rating,case_category_1,case_category_2
0,1,56,Nước ngon lắm,2020-01-03 20:34:00,5,Product,Drink
1,2,67,Máy lạnh nóng quá,2020-04-03 20:34:00,1,Facility,Air Conditioner
2,3,78,Bạn thu ngân rất dễ thương,2020-05-03 20:34:00,3,Service,Cashier
3,4,78,Hôm nay bánh mì mình ăn bị chua,2020-02-03 08:34:00,2,Product,Food
4,5,88,Không gian sạch sẽ,2025-07-01 10:00:00,5,Facility,Cleanliness
5,6,90,Nhân viên chậm chạp,2025-07-02 12:30:00,2,Service,Staff
6,7,77,Thức ăn rất ngon,2025-07-03 18:45:00,5,Product,Food
7,8,66,Quạt bị hỏng,2025-07-04 09:15:00,1,Facility,Fan
8,9,55,Nhạc hơi to,2025-07-05 20:00:00,3,Facility,Noise
9,10,44,Dịch vụ tuyệt vời,2025-07-06 13:30:00,5,Service,Experience


In [35]:
def transform_data(df):
    if df is None or df.empty:
        return None, None, None, None

    df = df.dropna(subset=['case_id'])
    df['comment'] = df['comment'].apply(lambda comment: comment.replace('\n', ''))
    df['case_created_at'] = pd.to_datetime(df['case_created_at'])

    #fact_case
    fact_cases = df.copy()
    fact_cases['ticket_date'] = fact_cases['case_created_at'].dt.date
    fact_cases['is_five_star'] = fact_cases['case_rating'] == 5

    #dim_users
    dim_users = fact_cases[['user_id']].drop_duplicates()
    dim_users['first_case_date'] = fact_cases.groupby('user_id')['case_created_at'].min().reindex(dim_users['user_id']).values
    dim_users['total_cases'] = fact_cases.groupby('user_id')['case_id'].count().reindex(dim_users['user_id']).values

    # Tạo dim_categories
    dim_categories = fact_cases[['case_category_1', 'case_category_2']].drop_duplicates()
    dim_categories['category_id'] = dim_categories.apply(
        lambda x: pd.util.hash_pandas_object(pd.Series([x['case_category_1'], x['case_category_2']])).iloc[0], axis=1
    )

    #dim_dates
    date_range = pd.date_range(start='2020-01-01', end=pd.Timestamp.today().date())
    dim_dates = pd.DataFrame({'date_day': date_range})
    dim_dates['day_of_week'] = dim_dates['date_day'].dt.dayofweek
    dim_dates['month'] = dim_dates['date_day'].dt.month
    dim_dates['year'] = dim_dates['date_day'].dt.year
    dim_dates['date_day'] = dim_dates['date_day'].dt.date


    return fact_cases, dim_users, dim_categories, dim_dates

fact_cases, dim_users, dim_categories, dim_dates = transform_data(df)

In [31]:
print("Fact Cases")
fact_cases

Fact Cases


,case_id,user_id,comment,case_created_at,case_rating,case_category_1,case_category_2,ticket_date,is_five_star
0,1,56,Nước ngon lắm,2020-01-03 20:34:00,5,Product,Drink,2020-01-03,True
1,2,67,Máy lạnh nóng quá,2020-04-03 20:34:00,1,Facility,Air Conditioner,2020-04-03,False
2,3,78,Bạn thu ngân rất dễ thương,2020-05-03 20:34:00,3,Service,Cashier,2020-05-03,False
3,4,78,Hôm nay bánh mì mình ăn bị chua,2020-02-03 08:34:00,2,Product,Food,2020-02-03,False
4,5,88,Không gian sạch sẽ,2025-07-01 10:00:00,5,Facility,Cleanliness,2025-07-01,True
5,6,90,Nhân viên chậm chạp,2025-07-02 12:30:00,2,Service,Staff,2025-07-02,False
6,7,77,Thức ăn rất ngon,2025-07-03 18:45:00,5,Product,Food,2025-07-03,True
7,8,66,Quạt bị hỏng,2025-07-04 09:15:00,1,Facility,Fan,2025-07-04,False
8,9,55,Nhạc hơi to,2025-07-05 20:00:00,3,Facility,Noise,2025-07-05,False
9,10,44,Dịch vụ tuyệt vời,2025-07-06 13:30:00,5,Service,Experience,2025-07-06,True


In [24]:
print("Dim users")
dim_users

Dim users


,user_id,first_case_date,total_cases
0,56,2020-01-03 20:34:00,1
1,67,2020-04-03 20:34:00,1
2,78,2020-02-03 08:34:00,2
4,88,2025-07-01 10:00:00,1
5,90,2025-07-02 12:30:00,1
6,77,2025-07-03 18:45:00,1
7,66,2025-07-04 09:15:00,1
8,55,2025-07-05 20:00:00,1
9,44,2025-07-06 13:30:00,1
10,33,2025-07-07 08:00:00,1


In [19]:
print("Dim categories")
dim_categories


Dim categories


,case_category_1,case_category_2,category_id
0,Product,Drink,4204908215465864965
1,Facility,Air Conditioner,966197678479132483
2,Service,Cashier,3051151608602800582
3,Product,Food,4204908215465864965
4,Facility,Cleanliness,966197678479132483
5,Service,Staff,3051151608602800582
7,Facility,Fan,966197678479132483
8,Facility,Noise,966197678479132483
9,Service,Experience,3051151608602800582
10,Facility,Seating,966197678479132483


In [20]:
print("Dim dates")
dim_dates

Dim dates


,date_day,day_of_week,month,year
0,2020-01-01,2,1,2020
1,2020-01-02,3,1,2020
2,2020-01-03,4,1,2020
3,2020-01-04,5,1,2020
4,2020-01-05,6,1,2020
...,...,...,...,...
2012,2025-07-05,5,7,2025
2013,2025-07-06,6,7,2025
2014,2025-07-07,0,7,2025
2015,2025-07-08,1,7,2025


In [36]:
def load_to_snowflake(fact_cases, dim_users, dim_categories, dim_dates):
    try:
        # Kết nối Snowflake
        conn = snowflake.connector.connect(
            account="********",
            user="duylnm",
            password="*****",
            warehouse="COMPUTE_WH",
            role="SYSADMIN",
            database='TEST',
            schema='PUBLIC'
        )
        cursor = conn.cursor()
        cursor.execute("USE WAREHOUSE COMPUTE_WH;")

        # Tạo bảng fact_cases
        cursor.execute("""
            CREATE OR REPLACE TABLE fact_cases (
                case_id INTEGER,
                user_id INTEGER,
                comment VARCHAR(1000),
                case_created_at TIMESTAMP,
                case_rating INTEGER,
                case_category_1 VARCHAR(50),
                case_category_2 VARCHAR(50),
                ticket_date DATE,
                is_five_star BOOLEAN
            ) CLUSTER BY (ticket_date);
        """)
        # Tạo bảng dim_users
        cursor.execute("""
            CREATE OR REPLACE TABLE dim_users (
                user_id INTEGER,
                first_case_date TIMESTAMP,
                total_cases INTEGER
            );
        """)
        # Tạo bảng dim_categories
        cursor.execute("""
            CREATE OR REPLACE TABLE dim_categories (
                category_id VARCHAR(50),
                case_category_1 VARCHAR(50),
                case_category_2 VARCHAR(50)
            );
        """)
        # Tạo bảng dim_dates
        cursor.execute("""
            CREATE OR REPLACE TABLE dim_dates (
                date_day DATE,
                day_of_week INTEGER,
                month INTEGER,
                year INTEGER
            );
        """)

        # Tải dữ liệu
        if fact_cases is not None and not fact_cases.empty:
            write_pandas(conn, fact_cases, 'FACT_CASES', quote_identifiers=False)
            print(f"Loaded {len(fact_cases)} rows to fact_cases")
        if dim_users is not None and not dim_users.empty:
            print("Loading dim_users")
            write_pandas(conn, dim_users, 'DIM_USERS', quote_identifiers=False)
            print(f"Loaded {len(dim_users)} rows to dim_users")
        if dim_categories is not None and not dim_categories.empty:
            print("Loading dim_categories")
            write_pandas(conn, dim_categories, 'DIM_CATEGORIES', quote_identifiers=False)
            print(f"Loaded {len(dim_categories)} rows to dim_categories")
        if dim_dates is not None and not dim_dates.empty:
            print("Loading dim_dates")
            write_pandas(conn, dim_dates, 'DIM_DATES', quote_identifiers=False)
            print(f"Loaded {len(dim_dates)} rows to dim_dates")

        conn.close()
    except Exception as e:
        print(f"Error loading to Snowflake: {e}")

load_to_snowflake(fact_cases, dim_users, dim_categories, dim_dates)

Loaded 14 rows to fact_cases
Loading dim_users


/var/folders/fq/f_p95hfx63j6_m9f116tk0p40000gn/T/ipykernel_28588/2926132048.py:62: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(conn, dim_users, 'DIM_USERS', quote_identifiers=False)


Loaded 13 rows to dim_users
Loading dim_categories


/var/folders/fq/f_p95hfx63j6_m9f116tk0p40000gn/T/ipykernel_28588/2926132048.py:66: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(conn, dim_categories, 'DIM_CATEGORIES', quote_identifiers=False)


Loaded 11 rows to dim_categories
Loading dim_dates
Loaded 2017 rows to dim_dates
